<a href="https://colab.research.google.com/github/tashrifmahmud/LLM-Project/blob/main/notebooks/1-preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will import multi_news dataset and use it for document summarization.

Accessing Dataset: alexfabbri/multi_news

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

In [3]:
ds = load_dataset('multi_news')

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

multi_news.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.src.cleaned:   0%|          | 0.00/548M [00:00<?, ?B/s]

train.tgt:   0%|          | 0.00/58.8M [00:00<?, ?B/s]

...

test.src.cleaned:   0%|          | 0.00/69.0M [00:00<?, ?B/s]

test.tgt:   0%|          | 0.00/7.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Checking our imported dataset.

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

We have 3 splits here: train, validation and test.

In [5]:
ds['train'][0]

{'document': 'National Archives \n \n Yes, it’s that time again, folks. It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. \n \n A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. \n \n Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you’re here, why don’t you sign up to follow us on Twitter. \n \n Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy 

In [6]:
ds['train'].features

{'document': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None)}

Now we will turn or dataset into pandas dataframe for data preprocessing.

In [7]:
import pandas as pd

In [8]:
df_train = ds['train'].to_pandas()
df_test = ds['test'].to_pandas()
df_validation = ds['validation'].to_pandas()

In [9]:
df_train.head()

,document,summary
0,"National Archives \n \n Yes, it’s that time ag...",– The unemployment rate dropped to 8.2% last m...
1,LOS ANGELES (AP) — In her first interview sinc...,"– Shelly Sterling plans ""eventually"" to divorc..."
2,"GAITHERSBURG, Md. (AP) — A small, private jet ...",– A twin-engine Embraer jet that the FAA descr...
3,Tucker Carlson Exposes His Own Sexism on Twitt...,– Tucker Carlson is in deep doodoo with conser...
4,A man accused of removing another man's testic...,– What are the three most horrifying words in ...


In [10]:
df_train.shape

(44972, 2)

We will do the following for cleaning text:
1. Removing punctuation
2. Remove newline characters and trailing spaces
3. Lowercasing characters
4. Removing stop words

In [11]:
import string

In [12]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [13]:
# We will define the remove_punctuation function
def remove_punctuation(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

In [14]:
# Now we will apply the function to each column in all dataframes
df_train['document'] = df_train['document'].apply(remove_punctuation)
df_train['summary'] = df_train['summary'].apply(remove_punctuation)

df_test['document'] = df_test['document'].apply(remove_punctuation)
df_test['summary'] = df_test['summary'].apply(remove_punctuation)

df_validation['document'] = df_validation['document'].apply(remove_punctuation)
df_validation['summary'] = df_validation['summary'].apply(remove_punctuation)

In [15]:
print(df_train.head())


                                            document  \
0  National Archives \n \n Yes it’s that time aga...   
1  LOS ANGELES AP — In her first interview since ...   
2  GAITHERSBURG Md AP — A small private jet has c...   
3  Tucker Carlson Exposes His Own Sexism on Twitt...   
4  A man accused of removing another mans testicl...   

                                             summary  
0  – The unemployment rate dropped to 82 last mon...  
1  – Shelly Sterling plans eventually to divorce ...  
2  – A twinengine Embraer jet that the FAA descri...  
3  – Tucker Carlson is in deep doodoo with conser...  
4  – What are the three most horrifying words in ...  


We will now remove newline characters and extra spaces.

In [16]:
# New function to clean text further
def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.strip()
    return text

In [17]:
# Apply the cleaning function to all columns
df_train['document'] = df_train['document'].apply(clean_text)
df_train['summary'] = df_train['summary'].apply(clean_text)

df_test['document'] = df_test['document'].apply(clean_text)
df_test['summary'] = df_test['summary'].apply(clean_text)

df_validation['document'] = df_validation['document'].apply(clean_text)
df_validation['summary'] = df_validation['summary'].apply(clean_text)

In [18]:
# Check the cleaned data
print(df_train.head())

                                            document  \
0  National Archives     Yes it’s that time again...   
1  LOS ANGELES AP — In her first interview since ...   
2  GAITHERSBURG Md AP — A small private jet has c...   
3  Tucker Carlson Exposes His Own Sexism on Twitt...   
4  A man accused of removing another mans testicl...   

                                             summary  
0  – The unemployment rate dropped to 82 last mon...  
1  – Shelly Sterling plans eventually to divorce ...  
2  – A twinengine Embraer jet that the FAA descri...  
3  – Tucker Carlson is in deep doodoo with conser...  
4  – What are the three most horrifying words in ...  


Now we will lowercase the texts in all dfs.

In [19]:
# Convert everything to lowercase
df_train['document'] = df_train['document'].str.lower()
df_train['summary'] = df_train['summary'].str.lower()

df_test['document'] = df_test['document'].str.lower()
df_test['summary'] = df_test['summary'].str.lower()

df_validation['document'] = df_validation['document'].str.lower()
df_validation['summary'] = df_validation['summary'].str.lower()


In [20]:
# Check the lowercase data
print(df_train.head())

                                            document  \
0  national archives     yes it’s that time again...   
1  los angeles ap — in her first interview since ...   
2  gaithersburg md ap — a small private jet has c...   
3  tucker carlson exposes his own sexism on twitt...   
4  a man accused of removing another mans testicl...   

                                             summary  
0  – the unemployment rate dropped to 82 last mon...  
1  – shelly sterling plans eventually to divorce ...  
2  – a twinengine embraer jet that the faa descri...  
3  – tucker carlson is in deep doodoo with conser...  
4  – what are the three most horrifying words in ...  


Since we will use a pretrained model like BART or T5 for comparison, we will not remove stop words for now as it can have unwanted effect.

Now we can save our df for now and work on them in notebook 2.

In [22]:
from datasets import Dataset, DatasetDict

In [23]:
# We will convert our 3 dfs to Dataset objects
train = Dataset.from_pandas(df_train)
test = Dataset.from_pandas(df_test)
validation = Dataset.from_pandas(df_validation)

In [25]:
# Create a DatasetDict to hold all splits
ds_clean = DatasetDict({
    'train': train,
    'test': test,
    'validation': validation
})

In [26]:
# Now we can check our new ds
print(ds_clean)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})


We can save ds_clean in pickle.

In [27]:
import pickle

# Save as pickle
with open('/content/ds_clean.pkl', 'wb') as f:
    pickle.dump(ds_clean, f)

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
